Test

In [25]:
import fastf1

qualifying = fastf1.get_session(2022, "Bahrain", "Q")
qualifying.load(weather=True)
race = fastf1.get_session(2022, "Bahrain", "R")
race.load(weather=True)

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.0]
req            INFO 	Using cached da

In [5]:
import pandas as pd

drivers = pd.unique(qualifying.laps["Driver"])
print(drivers)

['LEC' 'VER' 'SAI' 'PER' 'HAM' 'BOT' 'MAG' 'ALO' 'RUS' 'GAS' 'OCO' 'MSC'
 'NOR' 'ALB' 'ZHO' 'TSU' 'HUL' 'RIC' 'STR' 'LAT']


In [10]:
from fastf1.core import Laps
qualifying_times = list()
for driver in drivers:
    qualifying_times.append(qualifying.laps.pick_drivers(driver).pick_fastest())

qualifying_times = Laps(qualifying_times).sort_values(by="LapTime").reset_index(drop=True)
print(qualifying_times[["Driver", "LapTime"]])

   Driver                LapTime
0     LEC 0 days 00:01:30.558000
1     VER 0 days 00:01:30.681000
2     SAI 0 days 00:01:30.687000
3     PER 0 days 00:01:30.921000
4     HAM 0 days 00:01:31.048000
5     RUS 0 days 00:01:31.252000
6     MAG 0 days 00:01:31.461000
7     BOT 0 days 00:01:31.560000
8     ALO 0 days 00:01:31.621000
9     GAS 0 days 00:01:31.635000
10    OCO 0 days 00:01:31.782000
11    MSC 0 days 00:01:31.998000
12    NOR 0 days 00:01:32.008000
13    ZHO 0 days 00:01:32.493000
14    ALB 0 days 00:01:32.664000
15    TSU 0 days 00:01:32.750000
16    HUL 0 days 00:01:32.777000
17    RIC 0 days 00:01:32.945000
18    STR 0 days 00:01:33.032000
19    LAT 0 days 00:01:33.634000


In [94]:
for result in race.results.iterrows():
    print(result[1])

DriverNumber                                                         16
BroadcastName                                                 C LECLERC
Abbreviation                                                        LEC
DriverId                                                        leclerc
TeamName                                                        Ferrari
TeamColor                                                        ed1c24
TeamId                                                          ferrari
FirstName                                                       Charles
LastName                                                        Leclerc
FullName                                                Charles Leclerc
HeadshotUrl           https://www.formula1.com/content/dam/fom-websi...
CountryCode                                                         MON
Position                                                            1.0
ClassifiedPosition                                              

In [47]:
for result in race.results.iterrows():
    print(result[1]["Position"])

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0


Factors for ML Model to consider:

- Starting positions
- Gap to pole
- Teammate starting positions
- Weather (Wind, Rain, Average Track temp and air temp throughout full race)
- Retired
- Downforce index (may have to hardcode this) // for later
- Tyre wear index (may have to hardcode this) // for later

In [ ]:
# Calculate average wind direction

wind_speed = race.weather_data["WindSpeed"]
average_wind_speed = round(sum(wind_speed) / len(wind_speed), 5)


In [43]:
# Calculate average air temp

air_temp = race.weather_data["AirTemp"]
average_air_temp = round(sum(air_temp) / len(air_temp), 5)



In [44]:
# Calculate average track temp

track_temp = race.weather_data["TrackTemp"]
average_track_temp = round(sum(track_temp) / len(track_temp), 5)

In [93]:
# Calculate rain percentage
rain_data = race.weather_data["Rainfall"]
rain_count = 0

for rain in rain_data:
    rain_count = rain_count + 1 if rain else rain_count

rain_percentage = round(rain_count / len(rain_data) * 100, 5)

In [81]:
# Get gap to pole for each driver

pole_time = qualifying.laps.pick_fastest()["LapTime"]
pole_gaps = {}

for driver in qualifying.results.iterrows():
    driver = driver[1]
    if pd.isna(driver["Q3"]): 
        if pd.isna(driver["Q2"]):
            if pd.isna(driver["Q1"]):
                pole_gaps[driver["DriverNumber"]] = pd.NaT
            else:
                gap = driver["Q1"] - pole_time
                pole_gaps[driver["DriverNumber"]] = gap if gap.total_seconds() >= 0 else pd.NaT 
        else:
            gap = driver["Q2"] - pole_time
            pole_gaps[driver["DriverNumber"]] = gap if gap.total_seconds() >= 0 else pd.NaT
    else:
        gap = driver["Q3"] - pole_time
        pole_gaps[driver["DriverNumber"]] = gap if gap.total_seconds() >= 0 else pd.NaT

0 days 00:01:30.558000
{'16': Timedelta('0 days 00:00:00'), '1': Timedelta('0 days 00:00:00.123000'), '55': Timedelta('0 days 00:00:00.129000'), '11': Timedelta('0 days 00:00:00.363000'), '44': Timedelta('0 days 00:00:00.680000'), '77': Timedelta('0 days 00:00:01.002000'), '20': Timedelta('0 days 00:00:01.250000'), '14': Timedelta('0 days 00:00:01.637000'), '63': Timedelta('0 days 00:00:01.658000'), '10': Timedelta('0 days 00:00:01.780000'), '31': Timedelta('0 days 00:00:01.224000'), '47': Timedelta('0 days 00:00:01.440000'), '4': Timedelta('0 days 00:00:01.450000'), '23': Timedelta('0 days 00:00:02.106000'), '24': Timedelta('0 days 00:00:02.985000'), '22': Timedelta('0 days 00:00:02.192000'), '27': Timedelta('0 days 00:00:02.219000'), '3': Timedelta('0 days 00:00:02.387000'), '18': Timedelta('0 days 00:00:02.474000'), '6': Timedelta('0 days 00:00:03.076000')}


In [96]:
# Get teammate qualifying position for each driver

teammate_qualifying_pos = {}

for driver in qualifying.results.iterrows():
    driver = driver[1]
    team_id = driver["TeamId"]
    for teammate in qualifying.results.iterrows():
        teammate = teammate[1]
        if driver["DriverNumber"] == teammate["DriverNumber"]:
            continue

        if teammate["TeamId"] == team_id:
            teammate_qualifying_pos[driver["DriverNumber"]] = teammate["Position"]
            break

print(teammate_qualifying_pos)

{'16': 3.0, '1': 4.0, '55': 1.0, '11': 2.0, '44': 9.0, '77': 15.0, '20': 12.0, '14': 11.0, '63': 5.0, '10': 16.0, '31': 8.0, '47': 7.0, '4': 18.0, '23': 20.0, '24': 6.0, '22': 10.0, '27': 19.0, '3': 13.0, '18': 17.0, '6': 14.0}


In [88]:
# Get drivers own qualifying positions for each driver

driver_qualifying_pos = {}

for driver in qualifying.results.iterrows():
    driver = driver[1]

    driver_qualifying_pos[driver["DriverNumber"]] = driver["Position"]

print(driver_qualifying_pos)

{'16': 1.0, '1': 2.0, '55': 3.0, '11': 4.0, '44': 5.0, '77': 6.0, '20': 7.0, '14': 8.0, '63': 9.0, '10': 10.0, '31': 11.0, '47': 12.0, '4': 13.0, '23': 14.0, '24': 15.0, '22': 16.0, '27': 17.0, '3': 18.0, '18': 19.0, '6': 20.0}


CSV File columns:

- FullName
- Rain
- WindSpeed
- AirTemp
- TrackTemp
- QualifyingPos
- StartingPos
- GapToPole
- TeammateQualifyingPos
- Retired (True or false)

In [ ]:
import csv
# Write all data in a CSV file

fieldnames = [
        "FullName", "Season", "RoundNumber", "Rain", "WindSpeed", "AirTemp",
        "TrackTemp", "QualifyingPos", "StartingPos", "GapToPole", "TeammateQualifyingPos",
        "Retired"
    ]

with open("2022/bahrain_race.csv", "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for driver in race.results.iterrows():
        driver = driver[1]

        data = {
            "FullName" : driver["FullName"],
            "Rain" : rain_percentage,
            "WindSpeed" : average_wind_speed,
            "AirTemp" : average_air_temp,
            "TrackTemp" : average_track_temp,
            "QualifyingPos" : driver_qualifying_pos[driver["DriverNumber"]],
            "StartingPos" : driver["GridPosition"],
            "GapToPole" : pole_gaps[driver["DriverNumber"]],
            "TeammateQualifyingPos" : teammate_qualifying_pos[driver["DriverNumber"]],
            "Retired" : not (driver["Status"] == "Finished")
        }

        writer.writerow(data)


In [ ]:
from datetime import date

date_today = date.today()
for season in range(2022, 2026):
    for race in fastf1.get_event_schedule(season).iterrows():
        if race[1]["EventDate"] <= pd.Timestamp(date_today):
            pass

Spain
Bahrain
Sakhir
Jeddah
Melbourne
Imola
Miami
Barcelona
Monaco
Baku
Montréal
Silverstone
Spielberg
Le Castellet
Budapest
Spa-Francorchamps
Zandvoort
Monza
Marina Bay
Suzuka
Austin
Mexico City
São Paulo
Yas Island
Sakhir
Sakhir
Jeddah
Melbourne
Baku
Miami
Monaco
Barcelona
Montréal
Spielberg
Silverstone
Budapest
Spa-Francorchamps
Zandvoort
Monza
Marina Bay
Suzuka
Lusail
Austin
Mexico City
São Paulo
Las Vegas
Yas Island
Sakhir
Sakhir
Jeddah
Melbourne
Suzuka
Shanghai
Miami
Imola
Monaco
Montréal
Barcelona
Spielberg
Silverstone
Budapest
Spa-Francorchamps
Zandvoort
Monza
Baku
Marina Bay
Austin
Mexico City
São Paulo
Las Vegas
Lusail
Yas Island
Sakhir
Melbourne
Shanghai
Suzuka
Sakhir
Jeddah
Miami
Imola
Monaco
Barcelona
Montréal
Spielberg
Silverstone
